In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
from layers.noisy_bn import NoisyBatchNormalization

%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
fashion_mnist = keras.datasets.fashion_mnist

(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

In [3]:
train_images.shape

(60000, 28, 28)

In [4]:
# model 1
tf.keras.backend.clear_session()
input_node = keras.layers.Input((28,28))
expand = keras.layers.Lambda(lambda x: tf.expand_dims(tf.cast(x, tf.float32) / 255., axis=-1))(input_node)

conv1 = keras.layers.Conv2D(32, (3,3))(expand)
batch1 = keras.layers.BatchNormalization()(conv1)
pool1 = keras.layers.MaxPool2D((2,2))(batch1)

conv2 = keras.layers.Conv2D(32, (3,3))(pool1)
batch2 = keras.layers.BatchNormalization()(conv2)
pool2 = keras.layers.MaxPool2D((2,2))(batch2)

conv3 = keras.layers.Conv2D(32, (3,3))(pool2)
batch3 = keras.layers.BatchNormalization()(conv3)
pool3 = keras.layers.MaxPool2D((2,2))(batch3)

av1 = keras.layers.GlobalAveragePooling2D()(pool3)

h1 = keras.layers.Dense(512)(av1)
batch4 = keras.layers.BatchNormalization()(h1)
relu1 = keras.layers.Activation('relu')(batch4)

output_h = tf.keras.layers.Dense(10)(relu1)
output = tf.keras.layers.Activation('softmax')(output_h)

model1 = keras.Model(input_node, output)

In [5]:
model1.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

from tensorflow.keras.utils import to_categorical
model1.fit(train_images, to_categorical(train_labels), batch_size=32, epochs=1,validation_data=(test_images, to_categorical(test_labels)), verbose=1, shuffle=False)

Train on 60000 samples, validate on 10000 samples
60000/60000 [==============================] - 23s 377us/sample - loss: 0.5114 - accuracy: 0.8164 - val_loss: 0.4386 - val_accuracy: 0.8412


In [21]:
test_pre = keras.layers.Lambda(lambda x: tf.expand_dims(tf.cast(x, tf.float32) / 255., axis=-1))
test_conv = keras.layers.Conv2D(32, (3,3))

In [23]:
x = train_images[:3]
mid = test_pre(x)
out = test_conv(mid)

In [6]:
# model 2
tf.keras.backend.clear_session()
# input_node = keras.layers.Input((28,28))
# expand = keras.layers.Lambda(lambda x: tf.expand_dims(tf.cast(x, tf.float32) / 255., axis=-1))(input_node)

# conv1 = keras.layers.Conv2D(32, (3,3))(expand)
# batch1 = NoisyBatchNormalization(0)(conv1)
# pool1 = keras.layers.MaxPool2D((2,2))(batch1)

# conv2 = keras.layers.Conv2D(32, (3,3))(pool1)
# batch2 = NoisyBatchNormalization(0)(conv2)
# pool2 = keras.layers.MaxPool2D((2,2))(batch2)

# conv3 = keras.layers.Conv2D(32, (3,3))(pool2)
# batch3 = NoisyBatchNormalization(0)(conv3)
# pool3 = keras.layers.MaxPool2D((2,2))(batch3)

# av1 = keras.layers.GlobalAveragePooling2D()(pool3)

# h1 = keras.layers.Dense(512)(av1)
# batch4 = NoisyBatchNormalization(0)(h1)
# relu1 = keras.layers.Activation('relu')(batch4)

# output_h = tf.keras.layers.Dense(10, act)(relu1)
# output = tf.keras.layers.Activation('softmax')(output_h)


class PreProcess(keras.layers.Layer):
    def call(self, inputs):
        return tf.expand_dims(tf.cast(inputs, tf.float32) / 255., axis=-1)

class MyModel(keras.Model):
    def __init__(self, alpha):
        super(MyModel, self).__init__()
        
        self.pre_pro = PreProcess()
        
        self.conv1 = keras.layers.Conv2D(32, (3,3),data_format='channels_last')
        self.conv2 = keras.layers.Conv2D(32, (3,3))
        self.conv3 = keras.layers.Conv2D(32, (3,3))
        
        self.batch1 = NoisyBatchNormalization(alpha)
        self.batch2 = NoisyBatchNormalization(alpha)
        self.batch3 = NoisyBatchNormalization(alpha)
        self.batch4 = NoisyBatchNormalization(alpha)
        
        self.pool = keras.layers.MaxPool2D((2,2))
        
        self.globalav = keras.layers.GlobalAveragePooling2D()
        
        self.dense1 = keras.layers.Dense(512)
        self.dense2 = keras.layers.Dense(10, activation='softmax')
        self.relu1 = keras.layers.Activation('relu')
        
    def call(self, inputs, training=None):
        
        pro_inp = self.pre_pro(inputs)
        
        conv1 = self.conv1(pro_inp)
        batch1 = self.batch1(conv1, training)
        pool1 = self.pool(batch1)
        
        conv2 = self.conv2(pool1)
        batch2 = self.batch2(conv2, training)
        pool2 = self.pool(batch2)

        
        conv3 = self.conv3(pool2)
        batch3 = self.batch3(conv3, training)
        pool3 = self.pool(batch3)
        
        av = self.globalav(pool3)
        
        dense1 = self.dense1(av)
        batch4 = self.batch4(dense1, training)
        relu1 = self.relu1(batch4)
        
        pred = self.dense2(relu1)
        
        return pred
        


In [7]:
model = MyModel(alpha=0.)

In [9]:
_ = model(train_images[1:2])

[0, 1, 2]
[0, 1, 2]
[0, 1, 2]
[0]


In [8]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

from tensorflow.keras.utils import to_categorical
model.fit(train_images, to_categorical(train_labels), batch_size=32, epochs=1,validation_data=(test_images, to_categorical(test_labels)), verbose=1, shuffle=False)

Train on 60000 samples, validate on 10000 samples
60000/60000 [==============================] - 34s 566us/sample - loss: 0.5173 - accuracy: 0.8142 - val_loss: 1.8280 - val_accuracy: 0.3658


In [16]:
model1.layers[-4].moving_mean[:10], model.batch4.moving_variance[:10]

(<tf.Tensor: id=41824, shape=(10,), dtype=float32, numpy=
 array([ 0.4438476 , -0.5590519 , -0.24964793,  0.27279598,  0.62774837,
         0.59356534,  0.3737252 ,  0.01497004,  0.35746613,  0.4784101 ],
       dtype=float32)>,
 <tf.Tensor: id=41829, shape=(10,), dtype=float32, numpy=array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.], dtype=float32)>)

In [20]:
preds = np.zeros((10000, 10))

for i in range(int(10000 / 32) + 1):
    preds[i*32:(i+1)*32] = model(test_images[i*32:(i+1)*32], training=False).numpy()


In [16]:
preds = model.predict(test_images)

In [21]:
np.mean((np.argmax(preds, axis=-1) - test_labels) == 0)

0.5659

In [9]:
test_labels

array([9, 2, 1, ..., 8, 1, 5], dtype=uint8)

In [ ]:
tf.keras.layers.BatchNormalization()

In [16]:
class TestLayer(tf.keras.layers.Layer):
    def __init__(self, units, input_dim):
        super(TestLayer, self).__init__()
        self.w = self.add_weight(shape=(input_dim, units),
                             initializer=tf.zeros_initializer,
                             trainable=True)
        
        
    def call(self, inputs, training=None):
        self.w = self.w + 1
        return inputs

In [17]:
test_layer = TestLayer(32, 32)

In [18]:
input_const = tf.constant([1,2,3])
output_const = test_layer(input_const)

In [19]:
test_layer.w

<tf.Tensor: id=26, shape=(32, 32), dtype=float32, numpy=
array([[1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       ...,
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.]], dtype=float32)>